In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import tensorflow as tf
import p8_util
import p8_util_config

Using TensorFlow backend.


In [3]:
import p8_util
datadir = './data'

filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, feature_shape = p8_util.load_dataset(filename_dataset,is_label_encoded=True)

print("\nDataset : {}".format((x_train.shape, x_test.shape, y_train.shape, y_test.shape, nClasses, feature_shape)))

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 672, 224) (47, 672, 224) (414, 3) (47, 3)

Dataset : ((414, 672, 224), (47, 672, 224), (414,), (47,), 3, (672, 224))


In [4]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

In [ ]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (14147)kill: (14153)

#### Increase of train dataset against test dataset.

In [ ]:
OUTPUT_DIR

'./tmp/baseline'

In [ ]:
import p8_util
print("Input output dir= {}".format(OUTPUT_DIR))
oNNAdaNetBuilder = p8_util.create_nn_builder(feature_shape,OUTPUT_DIR)
oNNAdaNetBuilder.show()

Input output dir= ./tmp/baseline

 Number of convolutional layers= 2

*** NNAdaNetBuilder : NN Type=RNN

*** get_tf_head() : feature shape= (672, 224)

*** get_tf_head() : feature columns= [NumericColumn(key='images', shape=(672, 224), default_value=None, dtype=tf.float32, normalizer_fn=None)]

*** make_config() : output dir= ./tmp/baseline/RNN/SGRU

Max steps= 5 / Number of EPOCH=6


Adanet outputdir     : ............................ ./tmp/baseline/RNN
Adanet output log    : ............................ ./tmp/baseline/RNN/SGRU
NN type              : ............................ RNN
Features shape       : ............................ (672, 224)
Units in dense layer : ............................ 128
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 0.0
Weights initial

In [ ]:
p8_util_config.dict_adanet_config

{'adanet_feature_columns': None,
 'adanet_feature_shape': None,
 'adanet_tf_head': None,
 'adanet_lambda': 0.0,
 'adanet_is_learn_mixture_weights': True,
 'adanet_initial_num_layers': 1,
 'adanet_num_layers': None,
 'adanet_nn_candidate': 2,
 'adanet_output_dir': './tmp/adanet',
 'adanet_nn_layer_config': {'nn_type': 'RNN',
  'nn_dropout_rate': 0.0,
  'nn_batch_norm': True,
  'nn_dense_layer_num': 1,
  'nn_dense_unit_size': 128,
  'nn_logit_dimension': 3,
  'nn_optimizer': <tensorflow.python.training.adam.AdamOptimizer at 0x7f628e2e1940>,
  'nn_seed': 42,
  'nn_initializer_name': 'truncated_normal',
  'nn_layer_config': {'rnn_layer_num': 2,
   'rnn_hidden_units': 128,
   'rnn_timesteps': 224,
   'rnn_cell_type': 'SGRU'}}}

In [ ]:
import p8_util

batch_test = 20
input_fn_param={'num_epochs':p8_util_config.NUM_EPOCHS,\
                'batch_size':p8_util_config.BATCH_SIZE,\
                'feature_shape': oNNAdaNetBuilder.feature_shape,\
               }
train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        max_steps=p8_util_config.MAX_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        steps=None,\
        throttle_secs=1)

In [ ]:
params = {'net_builder': oNNAdaNetBuilder}
    
classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=oNNAdaNetBuilder.classifier_config)

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f628e199eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0512 17:58:10.804788 140063106246464 estimator.py:201] Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f628e199eb8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
import p8_util

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.


I0512 17:58:10.847403 140063106246464 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0512 17:58:10.848759 140063106246464 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.


...

INFO:tensorflow:Running local_init_op.


I0512 18:00:31.063090 140063106246464 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0512 18:00:31.213800 140063106246464 session_manager.py:493] Done running local_init_op.



*** generator() : labels shape= (414,) / label values= 1

*** generator() : labels shape= (414,) / label values= 1

*** generator() : labels shape= (414,) / label values= 1


In [ ]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")


In [ ]:
oNNAdaNetBuilder.output_dir_log

In [ ]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(oNNAdaNetBuilder.output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')